In [1]:
code_loc = "/home/shi/WorkSpace/projects/scMultiNet_workspace/"
root = "/home/shi/WorkSpace/projects/scMultiNet_Data/"
save_root = root + "/Step_1_data/Dataset_cook_removed/" #projects/scMultiNet_Data/Step_1_data/Dataset_cook_removed
import sys
sys.path.append(code_loc ) 

In [2]:
# start with a vocab list
# this related with the gene2vec model
#----> pre-trained part 
vocab_loc = code_loc +"/support_data/vocab_gene2vec_16906.pkl"
target_label = 'Ground_truth' # the label to be predicted

In [3]:

csv_path = root+"/Step_0_data/2024_05_19_Cook_emt_dataset_with_removal.h5ad"  #/home/shi/WorkSpace/projects/scMultiNet_Data/Step_0_data/2024_05_19_Cook_emt_dataset_with_removal.h5ad
data_path = root +  "/Step_0_data/2024_05_20_Cook_emt_dataset_only_removal.h5ad"


In [4]:
import anndata

adata = anndata.read_h5ad(csv_path)

In [5]:
adata.obs

,CellLine,stimulus,Experiment,pseudotimes,Ground_truth,Tissue
Mix1_AAACCTGAGGATGGAA-0,A549,EGF,Cook,0.48516449738804296,3d,Lung
Mix1_AAACCTGTCTATCCTA-0,A549,EGF,Cook,0.25346161999088135,8h_rm,Lung
Mix1_AAACGGGGTTCGTGAT-0,A549,EGF,Cook,0.6167297949786524,7d,Lung
Mix1_AAAGCAAGTTGTGGCC-0,A549,EGF,Cook,0.6523695007361208,8h_rm,Lung
Mix1_AAAGTAGCACTGTTAG-0,A549,EGF,Cook,0.34715971331363804,0d,Lung
...,...,...,...,...,...,...
Mix4b_TCTATTGAGATGTAAC-11,OVCA420,TNF,Cook,0.5429753869928555,3d_rm,Ovarian
Mix4b_TCTTTCCCAGCGTTCG-11,OVCA420,TNF,Cook,0.7734407184882754,3d,Ovarian
Mix4b_TGGCCAGTCATCATTC-11,OVCA420,TNF,Cook,0.7089686393043807,8h_rm,Ovarian
Mix4b_TGTGGTAGTGTCTGAT-11,OVCA420,TNF,Cook,0.6420790577610113,3d,Ovarian


In [6]:
# remove samples in 0d,8h,1d,3d,7d only keep 7d+8h('8h_rm'), 7d+1d ('1d_rm'), 7d+3d ('3d_rm')
adata = adata[~adata.obs["Ground_truth"].isin(["0d","8h","1d","3d","7d"])]

# remove samples not TGFb 
adata = adata[~adata.obs["stimulus"].isin(['EGF', 'TNF'])]

In [7]:
adata.shape

(1732, 11058)

In [8]:
#adata.write(data_path)

In [9]:
# check the data
import scanpy as sc
import pandas as pd
import numpy as np
adata = sc.read(data_path)

print(adata)
print(adata.obs.index)
print(adata.var.index)
print(f" negative values: {np.sum(adata.X<0)}")

AnnData object with n_obs × n_vars = 1732 × 11058
    obs: 'CellLine', 'stimulus', 'Experiment', 'pseudotimes', 'Ground_truth', 'Tissue'
    uns: 'source_paper'
    obsm: 'X_pca', 'X_umap', 'X_umap_pseudo'
Index(['Mix1_ACTTACTAGTGCGATG-1', 'Mix1_ATCATCTAGTAACCCT-1',
       'Mix1_CACAAACCACGGACAA-1', 'Mix1_CCAGCGACACCAGTTA-1',
       'Mix1_GCGCAACGTCGGATCC-1', 'Mix1_GGAAAGCCAACTTGAC-1',
       'Mix1_GGGAGATTCGCATGAT-1', 'Mix1_GTGTTAGAGTGCGATG-1',
       'Mix1_TCAGCTCAGTACGTAA-1', 'Mix1_TCGTACCAGTCCTCCT-1',
       ...
       'Mix4b_GCTTGAAAGGGTCGAT-10', 'Mix4b_GGGAGATAGTGGCACA-10',
       'Mix4b_GGGCATCTCCGTACAA-10', 'Mix4b_GGTGCGTTCATCACCC-10',
       'Mix4b_GTCCTCAAGGCCCTTG-10', 'Mix4b_GTGAAGGTCGAACGGA-10',
       'Mix4b_GTTACAGTCCTGCCAT-10', 'Mix4b_TAGGCATCAGGATTGG-10',
       'Mix4b_TCGAGGCAGTCCGTAT-10', 'Mix4b_TGGCCAGAGCGACGTA-10'],
      dtype='object', length=1732)
Index(['FO538757.2', 'AP006222.2', 'RP11-206L10.9', 'FAM41C', 'SAMD11',
       'NOC2L', 'KLHL17', 'PLEKHN1', 'HES4', 

In [10]:
# adata.X 有多少nan
import numpy as np
from scipy.sparse import issparse

# 假设 adata.X 是你的稀疏矩阵
# 首先，检查它是否为稀疏矩阵
if issparse(adata.X):
    # 将稀疏矩阵转换为密集格式，然后计算 NaN 值的数量
    nan_count = np.isnan(adata.X.toarray()).sum()
else:
    # 如果 adata.X 不是稀疏矩阵，直接计算 NaN 值的数量
    nan_count = np.isnan(adata.X).sum()

print(f"NaN values in adata.X: {nan_count} with shape {adata.X.shape}")


NaN values in adata.X: 0 with shape (1732, 11058)


In [11]:

from scLLM.Dataset.paras import Dataset_para
# define pre-processing by follow original implementation of scBERT

from scLLM.Dataset.paras import Dataset_para
# define pre-processing by follow original implementation of scBERT 
dataset_para_cls = Dataset_para(
                            var_idx=None,
                            obs_idx="Ground_truth",
                            vocab_loc=code_loc + "/Experiment/support_data/vocab_16k.json",
                            filter_gene_by_counts=False,
                            filter_cell_by_counts=0,
                            log1p=True,
                            log1p_base=2,

                            #
                            tokenize_name="scBERT",
                            cls_nb=5,
                            data_layer_name="X_log1p",
                            label_key = target_label,#"Ground_truth",#"Ground_truth",

                            test_size=None,#0.2, #use all data to inference
                            binarize=None, # not binarize use original label

                            )

dataset_para_reg = Dataset_para(
        vocab_loc=code_loc +"/Experiment/support_data/vocab_16k.json",
        var_idx = None,#"genes.gene_short_name",
        obs_idx="pseudotimes",
        filter_gene_by_counts=False,
        filter_cell_by_counts=200,
        log1p=True,
        log1p_base=2,

        tokenize_name="scBERT",
        cls_nb=1,
        data_layer_name="X_log1p",

        auto_map_str_labels=False,
        label_key = target_label,#"pseudotimes",

        test_size=None,#0.2, #use all data to inference
        binarize=None, # not binarize use original label for regression
    )

dataset_para = dataset_para_cls#dataset_para_cls
print(dataset_para)

Dataset_para(var_idx=None, obs_idx='Ground_truth', vocab_loc='/home/shi/WorkSpace/projects/scMultiNet_workspace//Experiment/support_data/vocab_16k.json', gene_vocab=None, use_key='X', filter_gene_by_counts=False, filter_cell_by_counts=0, normalize_total=10000.0, result_normed_key='X_normed', log1p=True, result_log1p_key='X_log1p', log1p_base=2, subset_hvg=False, hvg_use_key=None, hvg_flavor='seurat_v3', binning=None, result_binned_key='X_binned', tokenize_name='scBERT', return_pt=True, append_cls=True, include_zero_gene=False, cls_token='<cls>', max_len=16000, pad_token='<pad>', pad_value=-2, cls_appended=True, mask_ratio=0.15, mask_value=-1, preprocessed_loc=None, data_layer_name='X_log1p', label_key='Ground_truth', batch_label_key=None, cls_nb=5, binarize=None, bins=None, bin_min=None, bin_max=None, save_in_obs=True, auto_map_str_labels=True, map_dict=None, n_splits=1, test_size=None, random_state=2023, shuffle=True, sort_seq_batch=False)


In [12]:
# if this is the first time to run, need this block to init translate=True
# init preprocessor
from scLLM.Dataset.Reader import scReader
data_reader = scReader(dataset_para=dataset_para)
# init vocab
data_reader.init_vocab()

# load data
data_reader.load_adata(loc = data_path,translate=True)



/home/shi/anaconda3/envs/scLLM/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
scLLM - INFO - Initializing preprocessor ...
scLLM - INFO - use default vocab from dataset_para
scLLM - INFO - load vocab from /home/shi/WorkSpace/projects/scMultiNet_workspace//Experiment/support_data/vocab_16k.json
scLLM - INFO - Load data from anndata object.
scLLM - DEBUG - In original adata with gene 11058
scLLM - DEBUG - In original adata with gene 11058
scLLM - DEBUG - processing 0/16906
scLLM - DEBUG - processing 2000/16906
scLLM - DEBUG - processing 4000/16906
scLLM - DEBUG - processing 6000/16906
scLLM - DEBUG - processing 8000/16906
scLLM - DEBUG - processing 10000/16906
scLLM - DEBUG - processing 12000/16906
scLLM - DEBUG - processing 14000/16906
scLLM - DEBUG - processing 16000/16906
scLLM - INFO - create anndata 

In [13]:
data_reader.adata.X

<1732x16906 sparse matrix of type '<class 'numpy.float32'>'
	with 5809163 stored elements in Compressed Sparse Row format>

In [14]:
## preprocess
data_reader.preprocess()

scLLM - INFO - Preprocessing data with shape: (1732, 16906) ...
scLLM - INFO - Filtering cells by counts ...
scLLM - INFO - Filtered cells: 1732
scLLM - INFO - Normalizing total counts ...
scLLM - INFO - Log1p transforming ...
scLLM - INFO - Preprocessing finished.


In [15]:
print(data_reader.adata, data_reader.adata.X.shape)

AnnData object with n_obs × n_vars = 1732 × 16906
    obs: 'Ground_truth', 'n_counts'
    uns: 'log1p'
    layers: 'X_normed', 'X_log1p' (1732, 16906)


In [16]:
# get layer X_log1p
data = data_reader.adata.layers["X_log1p"] 
from scipy.sparse import coo_matrix, csr_matrix
# set nan to 0
if isinstance(data, csr_matrix):
    # 使用 CSR 格式的稀疏矩阵可以直接通过 data.data 访问非零元素
    data.data[np.isnan(data.data)] = 0
    # 注意: 这里没有改变矩阵的稀疏结构，只是将非零元素中的 NaN 值替换为 0
    data.data[np.isinf(data.data)] = data.data.max()
else:
    # 如果 data 不是稀疏矩阵，可以直接使用 numpy 的方法
    data[np.isnan(data)] = 0
    data[np.isinf(data)] = data.max()

if issparse(data):
    # 将稀疏矩阵转换为密集格式，然后计算 NaN 值的数量
    nan_count = np.isnan(data.toarray()).sum()
else:
    # 如果 adata.X 不是稀疏矩阵，直接计算 NaN 值的数量
    nan_count = np.isnan(data).sum()

print(f"NaN values in adata.X: {nan_count}")
data_reader.adata.layers["X_log1p"]  = data 

NaN values in adata.X: 0


In [17]:
# data <0 的个数
if issparse(data):
    # 将稀疏矩阵转换为密集格式，然后计算 NaN 值的数量
    nan_count = np.sum(data.toarray()<0)
else:
    # 如果 adata.X 不是稀疏矩阵，直接计算 NaN 值的数量
    nan_count = np.sum(data<0)

print(f"Negative values in adata.X: {nan_count}")

Negative values in adata.X: 0


In [18]:
print(data.shape)

(1732, 16906)


In [19]:
label_unique = data_reader.adata.obs[dataset_para.label_key].unique()
print(label_unique)
data_reader.adata.obs["str_"+dataset_para.label_key] = data_reader.adata.obs[dataset_para.label_key]

#
if data_reader.para.auto_map_str_labels:
    if data_reader.para.map_dict:
        print(data_reader.para.map_dict)
    else:
        data_reader.para.map_dict = {'8h_rm': 0,'1d_rm': 1,  '3d_rm': 2}
else:

    label_dict = {'8h_rm':0,'1d_rm':1,'3d_rm':2}
    # map obs label to int
    data_reader.adata.obs[dataset_para.label_key] = data_reader.adata.obs[dataset_para.label_key].map(label_dict)

['1d_rm', '8h_rm', '3d_rm']
Categories (3, object): ['1d_rm', '3d_rm', '8h_rm']


In [20]:

trainset,valset,weights = data_reader.postprocess()

# 输出数据集信息
print("trainset size: ",len(trainset))
print("valset size: ",len(valset)) if valset is not None else print("no valset")
#label_dict = {'1d_rm': 0, '8h_rm': 1, '3d_rm': 2}
print(data_reader.para.map_dict)
print("weights: ",weights)

scLLM - INFO - Map string labels to int automatically.
scLLM - INFO - Mapping from {'8h_rm': 0, '1d_rm': 1, '3d_rm': 2}


map_dict: {'8h_rm': 0, '1d_rm': 1, '3d_rm': 2}, name_list: ['1d_rm', '8h_rm', '3d_rm']


scLLM - INFO - Discritize label Ground_truth in obs_names


trainset size:  1732
no valset
{'8h_rm': 0, '1d_rm': 1, '3d_rm': 2}
weights:  None


In [21]:
trainset.sample_id = data_reader.adata.obs.index.to_list()
trainset.tissue_type = adata.obs["Tissue"].to_list()

In [22]:
print(len(trainset.sample_id), len(trainset.label), trainset.data.shape,trainset.sample_id[:5])

1732 1732 (1732, 16906) ['Mix1_ACTTACTAGTGCGATG-1', 'Mix1_ATCATCTAGTAACCCT-1', 'Mix1_CACAAACCACGGACAA-1', 'Mix1_CCAGCGACACCAGTTA-1', 'Mix1_GCGCAACGTCGGATCC-1']


In [23]:
trainset.label

tensor([[1],
        [1],
        [1],
        ...,
        [0],
        [1],
        [1]])

In [25]:
import dill
# 为trainset 添加其他labels

#target_task = f"/TrVal_dataset_GT_{target_stimulate}_{target_cellline}.pkl"
target_task = f"/TrVal_dataset_{target_label}.pkl"
loc = save_root + target_task
# 保存 trainset 到文件，并关联相应labels
with open(loc,"wb") as f:
    dill.dump([trainset,valset,weights,data_reader.para.map_dict],f)